<a href="https://colab.research.google.com/github/Murcha1990/ML_Course_PT/blob/main/Lecture4_Clustering%26MLSD/PT_Practice4_WhoIsTalking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Занятие 4. Who’s Talking? – Classify Encrypted TCP Traffic

Сегодня вам предстоит работать с реальным зашифрованным корпоративным сетевым трафиком.
Ваша задача — классифицировать, какое приложение или сервис сгенерировал данный TCP-поток, **несмотря на то, что полезная нагрузка зашифрована**, и доступны только размеры пакетов и их направления.

Датасет был собран в контролируемой корпоративной среде, где каждой сетевой сессии было присвоено соответствующее приложение или сервис.
Каждая запись представляет собой усечённый TCP-поток длиной до 30 пакетов и содержит только информацию о длине пакетов и направлении передачи.
Такой формат позволяет исследовать, как методы машинного обучения могут выявлять поведенческие паттерны в зашифрованном трафике без анализа его содержимого.

### Описание данных

Данные содержат размеченные примеры TCP-потоков с соответствующим приложением или сервисом (`app_service`).

**Колонки:**

* **app_service** — целевая переменная: название или числовой идентификатор приложения или сервиса, сгенерировавшего TCP-поток (например, Telegram, YouTube, Zoom)

* **tcp_len_1 – tcp_len_30** — последовательность длин до 30 TCP-пакетов внутри потока.
- Положительные значения соответствуют пакетам, отправленным клиентом.
- Отрицательные значения соответствуют пакетам, полученным от сервера.
-  Если сессия содержит меньше 30 пакетов, оставшиеся позиции заполняются нулями.

# <font color="green">Осознание целей и ограничений бизнеса - вопросы для обсуждения с заказчиком</font>

### Бизнес и цель

- Для чего нужна классификация трафика?

Примеры: мониторинг использования приложений, защита корпоративной сети, аудит использования SaaS, оптимизация пропускной способности.

- Какие приложения/сервисы нас интересуют? Только конкретные ключевые (Zoom, YouTube, Telegram), или все возможные?

- Какое требование к точности классификации? Например, достаточно 80% или нужно >95%?


### Данные и сбор

- Насколько репрезентативен датасет по сравнению с реальностью?

Примеры: различное время суток, разные сети (Wi-Fi, проводная), разные пользователи.

- Как часто появляются новые приложения или обновления существующих? Как быть с классами «новых» приложений?

- Есть ли ограничения на доступ к метаданным трафика, кроме длины пакетов и направления?


### Ограничения на модель

- Время отклика: нужен ли реальный онлайн детектор для каждого потока или можно пакетно/батчево?

- Ресурсы: допустимая нагрузка на CPU/GPU, память, скорость обработки.

- Частота обновления модели: сколько раз в неделю/месяц будем переобучать?


### Конфиденциальность и безопасность

- Данные трафика зашифрованы. Нужно ли дополнительно анонимизировать метаданные или учитывать регуляторные ограничения?

# <font color="green">Особенности данных и подготовка </font>

### Формат данных

- Последовательности TCP-пакетов длиной до 30, с положительными/отрицательными значениями.

- Пустые позиции заполнены нулями.

- Целевая переменная — app_service.


### Возможные pre-processing шаги

- Нормализация длин пакетов (например, деление на 1500 для приведения к [−1,1]).


### Дополнительные признаки

- Статистика по пакету: средняя длина, медиана, стандартное отклонение.

- Количество пакетов в каждом направлении (up/down).

- Интервалы между пакетами (если доступны временные метки).

- Можно использовать sequence models (RNN, LSTM, Transformer) или feature-based models (Random Forest, XGBoost).


### Особенности

- Потоки разной длины → padding до 30.

- Потоки сильно разрежены → много нулей.

- Много приложений → возможно сильный class imbalance.

# <font color="green">Бизнес и офлайн-метрики </font>

### Бизнес-метрики

- Доля правильно классифицированных приложений в онлайн-сценарии.

- Доля ошибочной классификации для критичных приложений (Zoom, корпоративные SaaS).

- Влияние на решения по безопасности и мониторингу (например, блокировка нежелательных сервисов).


### Офлайн ML-метрики

- Accuracy / Macro F1 / Weighted F1 (особенно при несбалансированных классах)

- Confusion matrix → какие приложения путаются чаще всего

- Top-K accuracy (например, Top-3, если нужно знать несколько вероятных кандидатов)

- ROC-AUC для каждого класса (multi-class AUC)

- Precision/Recall для ключевых приложений (например, высокочастотных или критичных)


### Особые моменты для офлайн-метрик

- Обратить внимание на имитацию реальной среды: train/test split должен быть по сессиям и пользователям, чтобы модель не запоминала конкретные шаблоны отдельных пользователей.

Сначала скачаем данные

In [ ]:
!wget -O response.json \
"https://cloud-api.yandex.net/v1/disk/public/resources/download?public_key=https://disk.yandex.ru/d/7LCvNsGL1mv90Q"

--2026-02-17 09:45:14--  https://cloud-api.yandex.net/v1/disk/public/resources/download?public_key=https://disk.yandex.ru/d/7LCvNsGL1mv90Q
Resolving cloud-api.yandex.net (cloud-api.yandex.net)... 213.180.204.127, 2a02:6b8::1:127
Connecting to cloud-api.yandex.net (cloud-api.yandex.net)|213.180.204.127|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 617 [application/json]
Saving to: ‘response.json’

response.json       100%[===================>]     617  --.-KB/s    in 0s      

2026-02-17 09:45:16 (37.1 MB/s) - ‘response.json’ saved [617/617]



In [ ]:
import json

with open("response.json") as f:
    data = json.load(f)

print(data["href"])

https://downloader.disk.yandex.ru/disk/aea8eaed46ea6eff189ebbf58b8f876daca0dece01abb7f55c3ea7c7622ce3e8/699470ec/fKqInKw3d7bLFOeFnMGnhOEt74e_AcHrfoeOiWQFqnxSx2KlJFe3moqi-rTFCO6djYu3q6EHAUP4X1BQu7yk7eyLDbl9lafVd4ruaDAPlxGr8npumZHI4midPdWhecNq?uid=0&filename=whos-talking-classify-the-app-by-its-packets.zip&disposition=attachment&hash=m8VLCd1u1YCGUik9dZVkJjbxXRNbrE0kS1p%2BEIBguw1klnm56L9GDaExZ6Yk0MeDq/J6bpmRyOJonT3VoXnDag%3D%3D%3A&limit=0&content_type=application%2Fzip&owner_uid=259847718&fsize=305374529&hid=11ebcd4213988e57481af625f75bf49e&media_type=compressed&tknv=v3


In [ ]:
!wget -O archive.zip "https://downloader.disk.yandex.ru/disk/aea8eaed46ea6eff189ebbf58b8f876daca0dece01abb7f55c3ea7c7622ce3e8/699470ec/fKqInKw3d7bLFOeFnMGnhOEt74e_AcHrfoeOiWQFqnxSx2KlJFe3moqi-rTFCO6djYu3q6EHAUP4X1BQu7yk7eyLDbl9lafVd4ruaDAPlxGr8npumZHI4midPdWhecNq?uid=0&filename=whos-talking-classify-the-app-by-its-packets.zip&disposition=attachment&hash=m8VLCd1u1YCGUik9dZVkJjbxXRNbrE0kS1p%2BEIBguw1klnm56L9GDaExZ6Yk0MeDq/J6bpmRyOJonT3VoXnDag%3D%3D%3A&limit=0&content_type=application%2Fzip&owner_uid=259847718&fsize=305374529&hid=11ebcd4213988e57481af625f75bf49e&media_type=compressed&tknv=v3"
# !wget -O archive.zip str(data["href"])

--2026-02-17 09:46:50--  https://downloader.disk.yandex.ru/disk/aea8eaed46ea6eff189ebbf58b8f876daca0dece01abb7f55c3ea7c7622ce3e8/699470ec/fKqInKw3d7bLFOeFnMGnhOEt74e_AcHrfoeOiWQFqnxSx2KlJFe3moqi-rTFCO6djYu3q6EHAUP4X1BQu7yk7eyLDbl9lafVd4ruaDAPlxGr8npumZHI4midPdWhecNq?uid=0&filename=whos-talking-classify-the-app-by-its-packets.zip&disposition=attachment&hash=m8VLCd1u1YCGUik9dZVkJjbxXRNbrE0kS1p%2BEIBguw1klnm56L9GDaExZ6Yk0MeDq/J6bpmRyOJonT3VoXnDag%3D%3D%3A&limit=0&content_type=application%2Fzip&owner_uid=259847718&fsize=305374529&hid=11ebcd4213988e57481af625f75bf49e&media_type=compressed&tknv=v3
Resolving downloader.disk.yandex.ru (downloader.disk.yandex.ru)... 77.88.21.127, 2a02:6b8::2:127
Connecting to downloader.disk.yandex.ru (downloader.disk.yandex.ru)|77.88.21.127|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://s02klg.storage.yandex.net/rdisk/aea8eaed46ea6eff189ebbf58b8f876daca0dece01abb7f55c3ea7c7622ce3e8/699470ec/fKqInKw3d7bLFOeFnMGnhOEt74e_Ac

In [ ]:
!unzip archive.zip -d data

Archive:  archive.zip
  inflating: data/sample_submission.csv  
  inflating: data/test.csv           
  inflating: data/train.csv          


Установим необходимые библиотеки

In [ ]:
!pip install catboost -q

import numpy as np
import pandas as pd
import gc

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from catboost import CatBoostClassifier, Pool

Загрузим данные и посмотрим на них

In [ ]:
df = pd.read_csv('/content/data/train.csv')

df.info()

/tmp/ipython-input-525312853.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/data/train.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8248546 entries, 0 to 8248545
Data columns (total 31 columns):
 #   Column       Dtype  
---  ------       -----  
 0   app_service  object 
 1   tcp_len_1    int64  
 2   tcp_len_2    float64
 3   tcp_len_3    float64
 4   tcp_len_4    float64
 5   tcp_len_5    float64
 6   tcp_len_6    float64
 7   tcp_len_7    float64
 8   tcp_len_8    float64
 9   tcp_len_9    float64
 10  tcp_len_10   float64
 11  tcp_len_11   float64
 12  tcp_len_12   float64
 13  tcp_len_13   float64
 14  tcp_len_14   float64
 15  tcp_len_15   float64
 16  tcp_len_16   float64
 17  tcp_len_17   float64
 18  tcp_len_18   float64
 19  tcp_len_19   float64
 20  tcp_len_20   float64
 21  tcp_len_21   float64
 22  tcp_len_22   float64
 23  tcp_len_23   float64
 24  tcp_len_24   float64
 25  tcp_len_25   float64
 26  tcp_len_26   float64
 27  tcp_len_27   float64
 28  tcp_len_28   float64
 29  tcp_len_29   float64
 30  tcp_len_30   float64
dtypes: float64(29), int64(1)

In [ ]:
df.sample(5)

,app_service,tcp_len_1,tcp_len_2,tcp_len_3,tcp_len_4,tcp_len_5,tcp_len_6,tcp_len_7,tcp_len_8,tcp_len_9,...,tcp_len_21,tcp_len_22,tcp_len_23,tcp_len_24,tcp_len_25,tcp_len_26,tcp_len_27,tcp_len_28,tcp_len_29,tcp_len_30
1961644,Ozon,1460,325.0,-1460.0,-1460.0,-1186.0,64.0,92.0,531.0,82.0,...,-1460.0,-1460.0,-1460.0,-1460.0,-1460.0,-1460.0,-1460.0,-1460.0,-1460.0,-1460.0
2421824,Teams,292,-99.0,363.0,-1460.0,-1460.0,-1116.0,74.0,-459.0,672.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5773973,moex-online,1380,342.0,-536.0,-536.0,-536.0,-536.0,-536.0,-536.0,-536.0,...,842.0,-13.0,-536.0,-536.0,-227.0,0.0,0.0,0.0,0.0,0.0
6605358,samsung-online,517,-1448.0,-1448.0,-1448.0,-1448.0,-433.0,80.0,998.0,-1208.0,...,39.0,24.0,-24.0,-24.0,0.0,0.0,0.0,0.0,0.0,0.0
4603041,google_maps,517,-1400.0,-1400.0,-391.0,64.0,-614.0,607.0,147.0,31.0,...,-1400.0,-1400.0,-1400.0,-1400.0,-1400.0,-1400.0,-1400.0,-1400.0,-1400.0,-1400.0


In [ ]:
df.memory_usage(deep=True).sum() / 1024**2

np.float64(2317.443407058716)

Казалось бы по определению длина - целая виличина, а тут используеться float64, давайте экономить память. Проверим диапозон значений, найдем есть ли наны или инфы, проверим количество не целых чисел

In [ ]:
target_col = "app_service"
feat_cols = [c for c in df.columns if c.startswith("tcp_len_")]

In [ ]:
stats = {
    "min_value": df[feat_cols].min().min(),
    "max_value": df[feat_cols].max().max(),
    "nan_count": np.isnan(df[feat_cols].values).sum(),
    "inf_count": np.isinf(df[feat_cols].values).sum(),
    "non_integer_count": np.sum((df[feat_cols].values % 1) != 0),
}

stats

{'min_value': -1464.0,
 'max_value': 1460.0,
 'nan_count': np.int64(0),
 'inf_count': np.int64(0),
 'non_integer_count': np.int64(0)}

Как видно значения по в диапозоне от -1464 до 1460 нет ни нанов ни инфов ни дробных чисел - изменим тип на int16

In [ ]:
feat_cols = [c for c in df.columns if c.startswith("tcp_len_")]
df[feat_cols] = df[feat_cols].astype(np.int16)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8248546 entries, 0 to 8248545
Data columns (total 31 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   app_service  object
 1   tcp_len_1    int16 
 2   tcp_len_2    int16 
 3   tcp_len_3    int16 
 4   tcp_len_4    int16 
 5   tcp_len_5    int16 
 6   tcp_len_6    int16 
 7   tcp_len_7    int16 
 8   tcp_len_8    int16 
 9   tcp_len_9    int16 
 10  tcp_len_10   int16 
 11  tcp_len_11   int16 
 12  tcp_len_12   int16 
 13  tcp_len_13   int16 
 14  tcp_len_14   int16 
 15  tcp_len_15   int16 
 16  tcp_len_16   int16 
 17  tcp_len_17   int16 
 18  tcp_len_18   int16 
 19  tcp_len_19   int16 
 20  tcp_len_20   int16 
 21  tcp_len_21   int16 
 22  tcp_len_22   int16 
 23  tcp_len_23   int16 
 24  tcp_len_24   int16 
 25  tcp_len_25   int16 
 26  tcp_len_26   int16 
 27  tcp_len_27   int16 
 28  tcp_len_28   int16 
 29  tcp_len_29   int16 
 30  tcp_len_30   int16 
dtypes: int16(30), object(1)
memory usage: 534.9+ MB


In [ ]:
df.memory_usage(deep=True).sum() / 1024**2

np.float64(901.4866428375244)

Почти в 4 раза экономнее, RAM в колабе скажет спасибо

In [ ]:
N = len(df)
N

8248546

In [ ]:
X0 = df[feat_cols]
y = df[target_col].astype(str)

sample_n = 1_000_000 # всё не влезет
idx = np.random.RandomState(42).choice(N, size=sample_n, replace=False)

X0 = X0.iloc[idx]
y  = y.iloc[idx]

Посмотрим на целевую переменную

In [ ]:
y.value_counts()

,count
app_service,
Telegram,12315
287,12277
GMail,12277
Wildberries,12268
Teams,12266
...,...
apteka-ru,135
appdynamics,133
MEGA,131


In [ ]:
set(y)

{'1',
 '1-password',
 '101-radio',
 '113',
 '120',
 '15',
 '218',
 '287',
 '299',
 '2gis',
 '343',
 '4',
 '93',
 'Amazon',
 'Anydesk',
 'Apple',
 'Baidu',
 'DataSaver',
 'DoH_DoT',
 'DropBox',
 'Facebook',
 'GMail',
 'Github',
 'Gitlab',
 'Instagram',
 'Ktalk',
 'LinkedIn',
 'Livejournal',
 'MEGA',
 'MSN',
 'Mattermost',
 'Odnoklassniki',
 'Ozon',
 'PlayStore',
 'Playstation',
 'QQ',
 'Quora',
 'Reddit',
 'Rutube',
 'Skype',
 'Snapchat',
 'SoundCloud',
 'Spotify',
 'Steam',
 'Teams',
 'Telegram',
 'TikTok',
 'Twitch',
 'Twitter',
 'WhatsApp',
 'Wikipedia',
 'Wildberries',
 'Yandex',
 'YouTube',
 'adblock-plus',
 'adobe',
 'aeroflot-online',
 'afisha-ru',
 'alfabank',
 'ali-wangwang-file-transfer',
 'alipay',
 'amd-online',
 'andata',
 'any-run',
 'anygo',
 'appdynamics',
 'apple-community',
 'apple-siri',
 'apple_icloud',
 'apple_push',
 'apteka-ru',
 'avito',
 'azure',
 'banki-ru',
 'beeline-online',
 'bing',
 'bitly',
 'blog-posting',
 'braintree',
 'bugsnag',
 'bybit',
 'ca-technolo

### Feature engineering

In [ ]:
X = X0.copy()

pos = (X0 > 0)
neg = (X0 < 0)
nonzero = (X0 != 0)

X["pkt_cnt"] = nonzero.sum(axis=1).astype(np.int16) # общее количество пакетов в потоке
X["n_up"]    = pos.sum(axis=1).astype(np.int16) # количество пакетов от клиента
X["n_down"]  = neg.sum(axis=1).astype(np.int16) # количество пакетов от сервера

X["bytes_up"]   = X0.where(pos, 0).sum(axis=1).astype(np.int32) # суммарное количество байт, отправленных клиентом
X["bytes_down"] = (-X0.where(neg, 0)).sum(axis=1).astype(np.int32) # суммарное количество байт, полученных от сервера

X.head()

,tcp_len_1,tcp_len_2,tcp_len_3,tcp_len_4,tcp_len_5,tcp_len_6,tcp_len_7,tcp_len_8,tcp_len_9,tcp_len_10,...,tcp_len_26,tcp_len_27,tcp_len_28,tcp_len_29,tcp_len_30,pkt_cnt,n_up,n_down,bytes_up,bytes_down
3899906,517,-1448,-1448,-1448,-23,64,992,-179,-62,-31,...,31,-305,-31,586,31,30,15,15,6043,5883
5090771,543,-399,51,234,-238,-1460,-1460,-1209,251,-1405,...,-1460,-7,249,-1405,-1359,30,8,22,2067,24073
6160203,517,-203,314,-1460,-349,-1136,250,-1391,-1137,250,...,0,0,0,0,0,13,4,9,1331,8226
3764889,517,-1448,-1448,-579,64,1208,56,-1208,-1448,-946,...,0,0,0,0,0,14,6,8,4412,8956
7759903,1460,647,-1460,-1460,-1176,-1287,64,92,548,-287,...,0,0,0,0,0,18,7,11,2894,9178


### Обучение модели и валидация

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

In [ ]:
model = CatBoostClassifier(
    loss_function="MultiClass",
    eval_metric="Accuracy",
    iterations=10,
    depth=5,
    learning_rate=0.15,
    random_seed=42,
    verbose=True,
    task_type="GPU",
    devices="0"
)

model.fit(
    X_train, y_train,
    eval_set=(X_val, y_val),
    use_best_model=True
)

pred = model.predict(X_val).reshape(-1)
print("val accuracy:", accuracy_score(y_val, pred))

0:	learn: 0.1354363	test: 0.1349750	best: 0.1349750 (0)	total: 3.8s	remaining: 34.2s
1:	learn: 0.2197900	test: 0.2191650	best: 0.2191650 (1)	total: 7.57s	remaining: 30.3s
2:	learn: 0.2714337	test: 0.2710950	best: 0.2710950 (2)	total: 11.4s	remaining: 26.6s
3:	learn: 0.2998125	test: 0.2995950	best: 0.2995950 (3)	total: 15.5s	remaining: 23.2s
4:	learn: 0.3157275	test: 0.3154300	best: 0.3154300 (4)	total: 19.1s	remaining: 19.1s
5:	learn: 0.3383762	test: 0.3377200	best: 0.3377200 (5)	total: 22.8s	remaining: 15.2s
6:	learn: 0.3558263	test: 0.3550100	best: 0.3550100 (6)	total: 26.4s	remaining: 11.3s
7:	learn: 0.4159288	test: 0.4152800	best: 0.4152800 (7)	total: 30.1s	remaining: 7.53s
8:	learn: 0.4330337	test: 0.4324200	best: 0.4324200 (8)	total: 33.7s	remaining: 3.75s
9:	learn: 0.4488475	test: 0.4475950	best: 0.4475950 (9)	total: 37.5s	remaining: 0us
bestTest = 0.447595
bestIteration = 9
val accuracy: 0.447595


# <font color="green">Внедрение и deployment </font>


### Онлайн vs оффлайн

- Онлайн: классификация каждого TCP-потока на лету → низкая задержка, нужна оптимизация модели.

- Оффлайн: анализ собранных логов → более тяжёлые модели, batch processing.


### Архитектура

- Data collector → потоковые данные о TCP → Preprocessing → Model → Results

- Возможна интеграция с корпоративным мониторингом.


### Выход модели

- Прямое приложение/сервис

- Top-K вероятности для учета неопределённости

- Возможность отбрасывать низкоуверенные предсказания.

# <font color="green">Логгирование и мониторинг </font>


### Что логировать

- Входные потоки (только метаданные, без payload)

- Предсказанные классы и вероятности

- Ошибки классификации (если есть метки)

- Время обработки


### Мониторинг

- Performance drift: точность со временем, особенно при обновлениях приложений

- Data drift: изменение распределения длины пакетов и направлений

- Модельный лог: уверенность предсказания, частота каждого класса

- Alerts: на резкие изменения трафика или появление неизвестных приложений


### Метрики производительности

- Latency per flow

- Throughput (flows/sec)

- Resource usage (CPU, RAM)